## Implementation of Auxiliary Classifiers (Inception v1 architecture (GoogleNet))

In [0]:
import numpy as np
from google.colab import files
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras import datasets
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, AveragePooling2D, MaxPooling2D, add, concatenate
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.utils import to_categorical

### Load CIFAR10 dataset

In [0]:
data = datasets.cifar10.load_data()
((x_train, y_train), (x_test, y_test)) = data
print(x_train.shape, x_test.shape)

170500096/170498071 [==============================] - 6s 0us/step
(50000, 32, 32, 3) (10000, 32, 32, 3)


In [0]:
x_train = x_train.astype('float') / 255
x_test = x_test.astype('float') / 255
num_classes = len(np.unique(y_train))
print("Number of classes = ", num_classes)

Number of classes =  10


In [0]:
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

### Model Construction

In [0]:
def inception_module(prev_layer):
    x1 = Conv2D(filters = 64, kernel_size = (1,1), padding = 'same')(prev_layer)
    x2 = Conv2D(filters = 64, kernel_size = (3,3), padding = 'same')(prev_layer)
    x3 = Conv2D(filters = 64, kernel_size = (5,5), padding = 'same')(prev_layer)
    x4 = MaxPooling2D(strides = (1,1),padding = 'same')(prev_layer)
    concat = concatenate([x1,x2,x3,x4] , axis = 3) 
    return concat



In [0]:
def auxiliary_output(prev_layer, numclasses):
    aux = AveragePooling2D(pool_size=(5,5), strides = (3,3), padding = 'same')(prev_layer)
    aux = Conv2D(filters = 128, kernel_size = (1,1), padding = 'same')(aux)
    aux = BatchNormalization()(aux)
    aux = Flatten()(aux)
    aux = Dense(1024, activation = 'relu')(aux)
    aux = Dense(num_classes, activation = 'softmax')(aux)
    return aux

In [0]:
input_img = Input(shape=x_train.shape[1:])
num_classes = 10

def with_aux(input_img, num_classes):
  conv1 = Conv2D(15, kernel_size = (3,3))(input_img)
  prev_layer = conv1

  aux_ouput = []
  for i in range(3):
      x = inception_module(prev_layer)
      x2 = MaxPooling2D( padding = 'same')(x)
      prev_layer = x2
      aux_ouput.append(auxiliary_output(prev_layer, num_classes))
      
  avgpool = AveragePooling2D( padding = 'same')(prev_layer)
  flat = Flatten()(avgpool)
  den1 = Dense(64, activation='relu')(flat)
  main_output = Dense(num_classes, activation = 'softmax')(den1)
  model = Model(inputs=[input_img], outputs=[main_output]+aux_ouput)
  model.summary()
  model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
  esCallback = EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=0,
                              verbose=0,
                              mode='min')
  history = model.fit(x_train,[y_train, y_train, y_train, y_train],
                epochs=50,
                validation_split=0.1,
                shuffle=True,verbose=1, callbacks = [esCallback])


  scores = model.evaluate(x_test, [y_test, y_test, y_test, y_test], verbose=1)
  print('Test loss:', scores[0])
  print('Test accuracy:', scores[1])

def without_aux(input_img, num_classes):
  conv1 = Conv2D(15, kernel_size = (3,3))(input_img)
  prev_layer = conv1

  aux_ouput = []
  for i in range(3):
      x = inception_module(prev_layer)
      x2 = MaxPooling2D( padding = 'same')(x)
      prev_layer = x2
      
  avgpool = AveragePooling2D( padding = 'same')(prev_layer)
  flat = Flatten()(avgpool)
  den1 = Dense(64, activation='relu')(flat)
  main_output = Dense(num_classes, activation = 'softmax')(den1)
  model = Model(inputs=[input_img], outputs=[main_output])
  model.summary()
  model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
  esCallback = EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=0,
                              verbose=0,
                              mode='min')
  history = model.fit(x_train,[y_train],
                epochs=50,
                validation_split=0.1,
                shuffle=True,verbose=1, callbacks = [esCallback])


  scores = model.evaluate(x_test, [y_test], verbose=1)
  print('Test loss:', scores[0])
  print('Test accuracy:', scores[1])

In [0]:
with_aux(input_img, num_classes)

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_85 (Conv2D)              (None, 30, 30, 15)   420         input_9[0][0]                    
__________________________________________________________________________________________________
conv2d_86 (Conv2D)              (None, 30, 30, 64)   1024        conv2d_85[0][0]                  
__________________________________________________________________________________________________
conv2d_87 (Conv2D)              (None, 30, 30, 64)   8704        conv2d_85[0][0]                  
____________________________________________________________________________________________

In [0]:
without_aux(input_img, num_classes)

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_98 (Conv2D)              (None, 30, 30, 15)   420         input_9[0][0]                    
__________________________________________________________________________________________________
conv2d_99 (Conv2D)              (None, 30, 30, 64)   1024        conv2d_98[0][0]                  
__________________________________________________________________________________________________
conv2d_100 (Conv2D)             (None, 30, 30, 64)   8704        conv2d_98[0][0]                  
____________________________________________________________________________________________

### Observation
    - Auxiliary output improves accuarcy from 0.724 to 0.84